In [1]:
%cd ..

C:\Users\kubak\Desktop\engineer-project\github\LanguageModule


C:\Users\kubak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import cohere
import os
import json

# Łączenie się z api cohere

In [3]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('./api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Pobieranie dokumentów

In [4]:
documents_path =  f"./data/decoded_pdfs/"

In [5]:
# Lista do przechowywania zawartości książek
documents = []

# Iteracja przez wszystkie pliki w folderze
for i, filename in enumerate(os.listdir(documents_path)):
    if filename.endswith(".json"):  # Sprawdzenie, czy plik ma rozszerzenie .json
        file_path = os.path.join(documents_path, filename)
        
        # Otwórz plik JSON i załaduj dane
        with open(file_path, "r", encoding="utf-8") as file:
            book_content = json.load(file)

        # Dodaj każdą stronę oddzielnie
        for page in book_content.values():
            documents.append(page)

        # Połącz zawartość wszystkich stron w jedną zmienną tekstową
        #full_text = " ".join(book_content.values())
        # Dodaj pełny tekst książki do listy documents
        #documents.append(full_text)

# RAG - łączenie dokumentów

In [6]:
# Funkcja do wyszukiwania dokumentów w zbiorze
def retrieve_relevant_documents(query, documents):
    response = co.rerank(
        model='rerank-english-v3.0',
        query=query,
        documents=documents,
        top_n=5  # Liczba dokumentów do zwrócenia
    )
    return response

# Funkcja do generowania odpowiedzi
def generate_response(query, context):
    response = co.generate(
        model="command",  #-xlarge-nightly",  # Model generowania Cohere
        prompt=f"Użytkownik pyta: {query}\n"
               f"Kontekst: {context}\n"
               f"Odpowiedz:",
        max_tokens=100,  # Maksymalna długość odpowiedzi
        temperature=0.7  # Stopień kreatywności odpowiedzi
    )
    return response.generations[0].text.strip()



In [7]:
# Przykład zapytania użytkownika
query = "Can you explain why it's good to do a hip dominant squat?"
query = "How to fix buttwink error?"

In [8]:
# Krok 1: Znalezienie najbardziej adekwatnego dokumentu
retrieval_result = retrieve_relevant_documents(query, documents)

if retrieval_result.results:
    most_relevant_document = retrieval_result.results[0].document
    print(f"Najbardziej adekwatny dokument: {most_relevant_document}")
else:
    print("Brak trafnych wyników dla zapytania.")
    most_relevant_document = None

Najbardziej adekwatny dokument: None


In [9]:
# Krok 2: Wygenerowanie odpowiedzi lub fallback
if most_relevant_document:
    response = generate_response(query, most_relevant_document)
else:
    response = "Przepraszam, nie znalazłem odpowiedniego dokumentu, ale mogę spróbować odpowiedzieć ogólnie."

# Wyświetlenie wyniku
print("Pytanie użytkownika:", query)
print("Odpowiedź:", response)

Pytanie użytkownika: How to fix buttwink error?
Odpowiedź: Przepraszam, nie znalazłem odpowiedniego dokumentu, ale mogę spróbować odpowiedzieć ogólnie.


# Embeding model

In [12]:
# Krojenie dokumentów
documents = documents[1:10]

# Tworzenie embeddingów dokumentów
embeddings = co.embed(
    model="embed-english-v3.0",
    texts=documents,
    input_type="search_document"  
).embeddings

In [13]:
import numpy as np

# Funkcja obliczania podobieństwa kosinusowego
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Wyszukiwanie najbliższego dokumentu
def search(query, documents, embeddings):
    # Generowanie embeddingu dla zapytania
    query_embedding = co.embed(
        texts=[query],
        model="embed-english-v3.0",
        input_type="search_query"
    ).embeddings[0]
    
    # Obliczanie podobieństw
    similarities = [cosine_similarity(query_embedding, doc_emb) for doc_emb in embeddings]
    best_match_index = np.argmax(similarities)
    return documents[best_match_index], similarities[best_match_index]

In [17]:
def generate_answer(query):
    # Wyszukiwanie kontekstu
    context, similarity = search(query, documents, embeddings)

    print("context", context)
    # Generowanie odpowiedzi na podstawie kontekstu
    prompt = f"Pytanie: {query}\nKontekst: {context}\nOdpowiedź:"
    response = co.generate(
        model="command-xlarge-nightly",
        prompt=prompt,
        max_tokens=50,
        temperature=0.7
    )
    return response.generations[0].text.strip()



In [18]:
# Testowanie
query = "What is low bar squat?"
answer = generate_answer(query)
print("Odpowiedź:", answer)

context Basic  Barbell  Training 

The  squat has been  the most important yet most poorly understood  exercise  in  the training 
arsenal  for  a very long  time.  The  full  range  of motion  exercise  known  as  the  squat  is  the  single 
most useful  exercise in  the weight room,  and  our  most  valuable  tool  for  building  strength,  power, 
and size (fig. 2-1). 

The squat is so effective an exercise because of the way it uses the muscles around  the core 
of the  body.  Much is  made  of core  strength,  and  fortunes  have  been  made  selling new ways  to 
train the core muscles.  A  correct squat  perfectly  balances  all the  forces around  the knees  and  the 
hips, using these  muscles in exactly the way the  skeletal biomechanics  are designed  for them  to  be 
used,  over  their  anatomically  full  range  of motion.  The  postural  muscles  of  the  lower  back,  the 
upper  back,  the  abdominals  and  lateral  trunk  muscles,  the  costal  (ribcage)  muscles,  a